# Acoustic Feature Extraction for AVF Analysis

This notebook extracts 47 acoustic features from WAV recordings of arteriovenous fistula sounds.

**Features extracted:**
- A: Time-domain features (4 features)
- B: Frequency-domain features (20 features)
- C: Literature-based features (4 features)
- D: Bio-acoustic features (3 features)
- E: Perceptual features / MFCCs (16 features)

In [ ]:
!pip install numpy scipy librosa pandas openpyxl -q

import numpy as np
import librosa
import pandas as pd
import os
import glob
import warnings
from google.colab import drive
warnings.filterwarnings('ignore')

drive.mount('/content/drive')

In [ ]:
FOLDER_PATH = "/path/to/your/wav/folder/"
OUTPUT_FILE = "acoustic_features_output.xlsx"
START_TIME = 1.0
END_TIME = 6.0

In [ ]:
def extract_time_domain_features(y, sr):
    features = {}
    features['A1_RMS_Energy'] = np.mean(librosa.feature.rms(y=y))
    features['A2_Zero_Crossing_Rate'] = np.mean(librosa.feature.zero_crossing_rate(y))
    peak_amp = np.max(np.abs(y))
    avg_power = np.mean(y**2)
    features['A3_Peak_to_Average_Power_Ratio'] = (peak_amp**2) / avg_power if avg_power > 0 else 0
    threshold = 0.2 * peak_amp
    active_samples = np.sum(np.abs(y) > threshold)
    features['A4_Active_Signal_Ratio'] = active_samples / len(y) if len(y) > 0 else 0
    return features


def extract_frequency_domain_features(y, sr, stft):
    features = {}
    freqs = librosa.fft_frequencies(sr=sr, n_fft=(stft.shape[0] - 1) * 2)
    mean_fft_over_time = np.mean(stft, axis=1)

    for i in range(8):
        start_freq, end_freq = i * 250, (i + 1) * 250
        band_mask = (freqs >= start_freq) & (freqs < end_freq)
        if np.any(band_mask):
            band_data = mean_fft_over_time[band_mask]
            features[f'B{5+i}_Mean_Amp_{start_freq}-{end_freq}Hz'] = np.mean(band_data)
            features[f'B{13+i}_Peak_Amp_{start_freq}-{end_freq}Hz'] = np.max(band_data)
        else:
            features[f'B{5+i}_Mean_Amp_{start_freq}-{end_freq}Hz'] = 0
            features[f'B{13+i}_Peak_Amp_{start_freq}-{end_freq}Hz'] = 0

    features['B21_Spectral_Centroid'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    features['B22_Spectral_Rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85))
    features['B23_Spectral_Flatness'] = np.mean(librosa.feature.spectral_flatness(y=y))
    f0, _, _ = librosa.pyin(y, fmin=20, fmax=400)
    valid_f0 = f0[~np.isnan(f0)]
    return features


def extract_literature_features(stft, freqs):
    features = {}
    mean_fft_over_time = np.mean(stft, axis=1)

    features['C25_TMP'] = np.sum(stft**2)
    features['C26_MF'] = freqs[np.argmax(mean_fft_over_time)]
    idx_200hz = (np.abs(freqs - 200)).argmin()
    features['C27_PSD200'] = mean_fft_over_time[idx_200hz]**2
    low_band_mask = (freqs >= 100) & (freqs <= 250)
    high_band_mask = (freqs >= 500) & (freqs <= 700)
    low_peak = np.max(mean_fft_over_time[low_band_mask]) if np.any(low_band_mask) else 0
    high_peak = np.max(mean_fft_over_time[high_band_mask]) if np.any(high_band_mask) else 0
    features['C28_HLPR'] = high_peak / low_peak if low_peak > 0 else 0
    return features


def extract_bio_acoustic_features(y, sr):
    features = {}

    f0 = librosa.yin(y, fmin=20, fmax=400)
    valid_f0 = f0[~np.isnan(f0) & (f0 > 0)]

    if len(valid_f0) > 1:
        f0_diff = np.abs(np.diff(valid_f0))
        mean_f0 = np.mean(valid_f0)
        features['D29_Jitter'] = np.mean(f0_diff) / (mean_f0 + 1e-8)
        features['B24_F0_Mean'] = mean_f0
    else:
        features['D29_Jitter'] = 0.0
        features['B24_F0_Mean'] = 0.0

    rms = librosa.feature.rms(y=y)[0]
    valid_rms = rms[rms > 0.01]
    if len(valid_rms) > 1:
        rms_diff = np.abs(np.diff(valid_rms))
        features['D30_Shimmer'] = np.mean(rms_diff) / (np.mean(valid_rms) + 1e-8)
    else:
        features['D30_Shimmer'] = 0.0

    try:
        correlation = np.correlate(y, y, mode='full')
        correlation = correlation[correlation.size // 2:]
        peak_search_range = slice(1, min(len(correlation), sr // 20))
        if peak_search_range.stop > peak_search_range.start:
            harmonic_power = np.max(correlation[peak_search_range])
        else:
            harmonic_power = 0
        total_power = correlation[0]
        if total_power > harmonic_power > 0:
            noise_power = total_power - harmonic_power
            features['D31_HNR'] = 10 * np.log10(harmonic_power / (noise_power + 1e-8))
        else:
            features['D31_HNR'] = 0.0
    except:
        features['D31_HNR'] = 0.0

    return features


def extract_advanced_features(y, sr, stft):
    features = {}
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(13):
        features[f'E{32+i}_MFCC_{i+1}'] = np.mean(mfccs[i, :])
    features['E45_Spectral_Bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    features['E46_Spectral_Contrast'] = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr, n_bands=4))
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features['E47_Chroma_Vector_Variance'] = np.mean(np.var(chroma, axis=1))
    return features

In [ ]:
def analyze_all_features_from_file(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)

        start_sample = int(sr * START_TIME)
        end_sample = int(sr * END_TIME)
        y_cut = y[start_sample:end_sample]

        if len(y_cut) == 0:
            print(f"Warning: No audio data in specified range for {os.path.basename(file_path)}")
            return None

        stft = np.abs(librosa.stft(y_cut))
        freqs = librosa.fft_frequencies(sr=sr, n_fft=(stft.shape[0] - 1) * 2)

        all_features = {"File": os.path.basename(file_path)}
        all_features.update(extract_time_domain_features(y_cut, sr))
        all_features.update(extract_frequency_domain_features(y_cut, sr, stft))
        all_features.update(extract_literature_features(stft, freqs))
        all_features.update(extract_bio_acoustic_features(y_cut, sr))
        all_features.update(extract_advanced_features(y_cut, sr, stft))

        return all_features

    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")
        return None

In [ ]:
if __name__ == "__main__":
    wav_files = glob.glob(os.path.join(FOLDER_PATH, '*.wav'))
    all_results = []

    if not wav_files:
        print(f"No WAV files found in '{FOLDER_PATH}'. Please check the path.")
    else:
        print(f"Processing {len(wav_files)} files...")
        for file_path in wav_files:
            results = analyze_all_features_from_file(file_path)
            if results:
                all_results.append(results)

        if all_results:
            df = pd.DataFrame(all_results)
            df.to_excel(OUTPUT_FILE, index=False)
            print(f"\nProcessing complete. Results saved to '{OUTPUT_FILE}'.")
            try:
                from google.colab import files
                files.download(OUTPUT_FILE)
            except ImportError:
                print("File saved to current directory.")